In [1]:
%pip install --upgrade langchain langsmith langgraph langchain_openai openai python-dotenv javascript numpy genson
# run this in terminal: python3 -m javascript --install @near-lake/primitives
%reload_ext autoreload
%autoreload 2

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 975 kB 2.8 MB/s eta 0:00:01
     |████████████████████████████████| 127 kB 88.7 MB/s eta 0:00:01
     |████████████████████████████████| 88 kB 14.3 MB/s eta 0:00:01
     |████████████████████████████████| 40 kB 24.4 MB/s eta 0:00:01
     |████████████████████████████████| 327 kB 69.7 MB/s eta 0:00:01
     |████████████████████████████████| 5.2 MB 28.7 MB/s eta 0:00:01
     |████████████████████████████████| 332 kB 86.0 MB/s eta 0:00:01
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.1.73
    Uninstalling langsmith-0.1.73:
      Successfully uninstalled langsmith-0.1.73
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.2.4
    Uninstalling langchain-core-0.2.4:
      Successfully uninstalled langchain-core-0.2.4
  Attempting uninstall: openai
    Found existing installation: openai 1.31.1
    Uninstall

In [2]:
from dotenv import load_dotenv
import os
import sys
sys.path.append('..')
from langchain_core.messages import HumanMessage
from graph.trivial_agent_graph import IndexerAgentGraphBuilder
from agents.IndexerAgent import indexer_agent_model, tool_executor
from tools.JavaScriptRunner import tool_js_on_block_schema_func, tool_js_on_block_schema
from agents.BlockExtractorAgent import run_js_on_block_only_schema
import json

# Load .env file
load_dotenv('.env')

# Set model variables
OPENAI_BASE_URL = "https://api.openai.com/v1"
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_ORGANIZATION = os.getenv("OPENAI_ORGANIZATION")


os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = "Indexer Agent Local"


/Users/spark_near/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


# Execute Data Extractor Agent

In [14]:
from tools.NearLake import tool_get_block_heights
from agents.BlockExtractorAgent import block_extractor_agent_model

inputs = {
    "messages": [
        HumanMessage(
            content="Extract all FunctionCalls where receiverId is 'app.nearcrowd.near'. For each function call, I need signerId, block height, receiptId, block datetime, methodName. Also add all fields from args that are decoded from base64-encoded JSON. I only need actions for successful receipts.",
        )
    ]
}

# Define the tools that the agent will use
tools = [tool_js_on_block_schema_func, tool_get_block_heights]
model = block_extractor_agent_model(tools)

builder = IndexerAgentGraphBuilder(model, tool_executor(tools))
app = builder.graph()

js_code_res = app.with_config({"run_name": "TrivialGraph block_extractor_agent_model"}).invoke(inputs)

BadRequestError: Error code: 400 - {'error': {'message': "Invalid 'messages[7].tool_calls[0].function.name': string does not match pattern. Expected a string that matches the pattern '^[a-zA-Z0-9_-]+$'.", 'type': 'invalid_request_error', 'param': 'messages[7].tool_calls[0].function.name', 'code': 'invalid_value'}}

In [13]:
from langchain_utils import get_tool_call_arguments
from agents.BlockExtractorAgent import JsResponse

js_response = get_tool_call_arguments(js_code_res['messages'], JsResponse)

js_response.js_schema

ValidationError: 1 validation error for JsResponse
__root__
  Expecting ',' delimiter: line 2 column 1068 (char 1069) (type=value_error.jsondecode; msg=Expecting ',' delimiter; doc={
"js": "function extractData(block) {\n  const actions = block.actions();\n  const receipts = block.receipts();\n  const header = block.header();\n  const successfulReceipts = receipts.filter(receipt => receipt.status.SuccessValue !== undefined);\n  const functionCalls = actions.filter(action => action.operations.some(op => op.FunctionCall !== undefined && action.receiverId === 'app.nearcrowd.near'));\n  const result = functionCalls.map(action => {\n    const receipt = successfulReceipts.find(receipt => receipt.receiptId === action.receiptId);\n    if (receipt) {\n      const functionCall = action.operations.find(op => op.FunctionCall !== undefined);\n      return {\n        signerId: action.signerId,\n        blockHeight: header.height,\n        receiptId: action.receiptId,\n        blockDatetime: new Date

In [ ]:
from agents.DDLAgent import ddl_generator_agent_model

model = ddl_generator_agent_model()

inputs = {
    "messages": [
        HumanMessage(
            content="Here is the schema from JS developer: " + js_response.js_schema,
        )
    ]
}
builder = IndexerAgentGraphBuilder(model, tool_executor(tools))
app = builder.graph()

ddl_code_res = app.with_config({"run_name": "Graph ddl_generator_agent_model", "recursion_limit": 2}).invoke(inputs)


In [ ]:
ddl_code_res

In [ ]:
from agents.DDLAgent import DDLAgentResponse

ddl_response = get_tool_call_arguments(ddl_code_res['messages'], DDLAgentResponse)
print(ddl_response)

# Scratchpad
Cells below are for testing and debugging

In [ ]:
from tools.JavaScriptRunner import fetch_block, get_function_calls_from_block
from tools.contract_block_heights import get_block_heights

block_heights = get_block_heights("pool.near", 5)

block_heights

get_function_calls_from_block(block_heights[0], 'pool.near')


In [ ]:

from tools.JavaScriptRunner import run_js_on_block_only_schema, run_js_on_block
code = """
function extractData(block) {
    const actions = block.actions();
    const receipts = block.receipts();
    const header = block.header();

    const successfulReceipts = receipts.filter(receipt => receipt.status.SuccessValue);
    const filteredActions = actions.filter(action => action.receiverId === 'app.nearcrowd.near' && action.operations.some(op => op.FunctionCall));

    const result = [];

    for (const action of filteredActions) {
      for (const operation of action.operations) {
        if (operation.FunctionCall) {
          const receipt = receipts.find(receipt => receipt.receiptId === action.receiptId);
          if (receipt) {
            const args = JSON.parse(atob(operation.FunctionCall.args));
            result.push({
              signerId: action.signerId,
              blockHeight: header.height,
              receiptId: action.receiptId,
              receipt: receipt,
              blockDatetime: new Date(parseInt(header.timestampNanosec) / 1000000),
              methodName: operation.FunctionCall.methodName,
              ...args
            });
          }
        }
      }
    }

    return result;
  }
  return extractData(block);
"""
print(json.dumps(run_js_on_block(119688212, code)))

In [7]:
from tools.JavaScriptRunner import tool_infer_schema_of_js

schema = tool_infer_schema_of_js.run({
    "receiver": 'app.nearcrowd.near', 
    "js": 'return block.receipts()', 
    "from_days_ago": 1})

print(schema)

Getting block heights from bitmap indexer for receiver=app.nearcrowd.near from_days_ago=1 limit=10
Inferring schema for return block.receipts() on block height 120381999
Inferring schema for return block.receipts() on block height 120382000
{
  "type": "array",
  "items": {
    "type": "object",
    "properties": {
      "receiptKind": {
        "type": "string"
      },
      "receiptId": {
        "type": "string"
      },
      "receiverId": {
        "type": "string"
      },
      "predecessorId": {
        "type": "string"
      },
      "status": {
        "type": "object",
        "properties": {
          "SuccessValue": {
            "type": "string"
          },
          "SuccessReceiptId": {
            "type": "string"
          },
          "Failure": {
            "type": "object",
            "properties": {
              "ActionError": {
                "type": "object",
                "properties": {
                  "index": {
                    "type": "integer"